In [1]:
import requests
import json
import pandas as pd
from utility import date_to_timestamp,epoch_to_timestamp, timestamp_to_epoch
def get_validator_income_detail_history(validator_index, latest_epoch):
    """
    get the income detail history of a validator for the previous 100 epochs by using the beaconcha.in API
    """
    url = f'https://beaconcha.in/api/v1/validator/{validator_index}/incomedetailhistory?latest_epoch={latest_epoch}'
    response = requests.get(url)
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return None
def process_income_detail_history(res):
    """
    Porcess the income detail history of a validator
    """
    tt1=pd.DataFrame(res['data'])
    tt1['attestation_source_reward'] = tt1['income'].apply(lambda x: x['attestation_source_reward'] if 'attestation_source_reward' in x else 0)
    tt1['attestation_target_reward'] = tt1['income'].apply(lambda x: x['attestation_target_reward'] if 'attestation_target_reward' in x else 0)
    tt1['attestation_head_reward'] = tt1['income'].apply(lambda x: x['attestation_head_reward'] if 'attestation_head_reward' in x else 0)
    tt1['other_reward'] = tt1['income'].apply(lambda x: sum([int(x[i]) if i not in ['attestation_source_reward','attestation_target_reward','attestation_head_reward'] and 'penalty' not in i else 0 for i in x.keys()]))
    tt1['penalty'] = tt1['income'].apply(lambda x: sum([int(x[i]) if 'penalty' in i else 0 for i in x.keys()]))
    return tt1


In [3]:
def get_one_day_reward_one_validator(validator_index,date):
    #Get the earliest timestamp for the given date, where the date is a string '2022-10-30', and the earliest timestamp is '2022-10-30 00:00:00'.
    start_timestamp=date_to_timestamp(date)
    #Get the latest timestamp of the day for the given date string '2022-10-30', which is 2022-10-30 23:59:59.
    end_timestamp=date_to_timestamp(date,latest=True)
    start_epoch=int(timestamp_to_epoch(start_timestamp)+1)
    end_epoch=int(timestamp_to_epoch(end_timestamp))
    print(start_epoch,end_epoch)
    validator_rewards_total=pd.DataFrame()
    for epoch in range(end_epoch,start_epoch-100,-100):
        income_detail_history=get_validator_income_detail_history(validator_index,epoch)
        if income_detail_history and start_epoch-epoch<100:
            temp=process_income_detail_history(income_detail_history)
            validator_rewards_total=pd.concat([validator_rewards_total,temp])
        else:
            print('no data')
            break
    validator_rewards_total.drop(columns=['income','week','week_start','week_end'], inplace=True)  
    validator_rewards_total=validator_rewards_total.drop_duplicates()
    validator_rewards_total['time']=validator_rewards_total['epoch'].map(epoch_to_timestamp)
    validator_rewards_total['time']=pd.to_datetime(validator_rewards_total['time'],unit='s')
    validator_rewards_total['date']=validator_rewards_total['time'].dt.date
    validator_rewards_total=validator_rewards_total.sort_values(by=['epoch'])
    validator_rewards_total=validator_rewards_total[(validator_rewards_total['date']==pd.to_datetime(date).date())]
    validator_rewards_total['total_attestation_reward']=validator_rewards_total['attestation_source_reward']+validator_rewards_total['attestation_target_reward']+validator_rewards_total['attestation_head_reward']
    validator_rewards_total['total_reward']=validator_rewards_total['total_attestation_reward']+validator_rewards_total['other_reward']-validator_rewards_total['penalty']
    return validator_rewards_total   
# get_one_day_reward_one_validator(123356,'2022-10-30')

In [4]:
import random
#randomly select 10 validator in ramdon 10 days
validator_index_list=random.sample(range(0, 100000), 10)
date_list=random.sample(pd.date_range('2022-09-15','2023-04-05').strftime('%Y-%m-%d').tolist(), 10)
validator_index_data=dict(zip(validator_index_list,date_list))
reward_verification_data=pd.DataFrame()
for validator_index,date in validator_index_data.items():
    res=get_one_day_reward_one_validator(validator_index,date)
    reward_verification_data=pd.concat([reward_verification_data,res])
    reward_verification_data_daily=reward_verification_data.groupby(['date','validatorindex']).agg({'total_reward':'sum','total_attestation_reward':'sum','other_reward':'sum','penalty':'sum'}).reset_index()
    
   

156263 156487
176288 176512
157388 157612
184163 184387
159413 159637
156938 157162
179888 180112
185063 185287
173813 174037
160763 160987


In [5]:
reward_verification_data_daily

,date,validatorindex,total_reward,total_attestation_reward,other_reward,penalty
0,2022-10-27,89175,3147217,3150941,0,3724
1,2022-10-30,49579,3137206,3144096,0,6890
2,2022-11-01,59176,3126813,3126813,0,0
3,2022-11-10,11029,2467138,2702585,0,235447
4,2022-11-16,65067,3098273,3101927,0,3654
5,2023-01-13,14720,2993459,3000037,0,6578
6,2023-01-24,49827,2992321,2998860,0,6539
7,2023-02-09,97351,2962964,2969438,0,6474
8,2023-02-28,46616,2934895,2934895,0,0
9,2023-03-04,69692,2913092,2913092,0,0


## query rewards of these validators in these days in our data

In [6]:
file='/local/scratch/exported/Ethereum_token_txs_data_TY_23/rewards/eth2_reward_ether/daily_validator_index_reward/proposer_attestation_sync_daily_reward_0.parquet'
reward_data=pd.read_parquet(file)
reward_data['date']=pd.to_datetime(reward_data['date']).dt.date

In [8]:
df_verification_data=pd.DataFrame()
date_validator=zip(reward_verification_data_daily['date'],reward_verification_data_daily['validatorindex'])
for date,validatorindex in date_validator:
    date=pd.to_datetime(date).date()
    # print(date,validatorindex)
    temp=reward_data[(reward_data['date']==date) & (reward_data['validator_index']==validatorindex)]
    df_verification_data=pd.concat([df_verification_data,temp])

## merge our data and the data from the beaconcha.in

In [9]:
df_verification_data1=pd.merge(df_verification_data,reward_verification_data_daily,left_on=['date','validator_index'],right_on=['date','validatorindex'],how='left')
df_verification_data1['total_attestation_reward']=df_verification_data1['total_attestation_reward']-df_verification_data1['penalty']
df_verification_data1

,date,validator_index,Total reward,Attestation reward,Proposer reward,Sync committee reward,validatorindex,total_reward,total_attestation_reward,other_reward,penalty
0,2022-10-27,89175.0,3147217.0,3147217.0,0.0,0.0,89175,3147217,3147217,0,3724
1,2022-10-30,49579.0,3137206.0,3137206.0,0.0,0.0,49579,3137206,3137206,0,6890
2,2022-11-01,59176.0,3126813.0,3126813.0,0.0,0.0,59176,3126813,3126813,0,0
3,2022-11-10,11029.0,2467138.0,2467138.0,0.0,0.0,11029,2467138,2467138,0,235447
4,2022-11-16,65067.0,3098273.0,3098273.0,0.0,0.0,65067,3098273,3098273,0,3654
5,2023-01-13,14720.0,2993459.0,2993459.0,0.0,0.0,14720,2993459,2993459,0,6578
6,2023-01-24,49827.0,2992321.0,2992321.0,0.0,0.0,49827,2992321,2992321,0,6539
7,2023-02-09,97351.0,2962964.0,2962964.0,0.0,0.0,97351,2962964,2962964,0,6474
8,2023-02-28,46616.0,2934895.0,2934895.0,0.0,0.0,46616,2934895,2934895,0,0
9,2023-03-04,69692.0,2913092.0,2913092.0,0.0,0.0,69692,2913092,2913092,0,0
